In [1]:
import os
import yaml
import numpy as np
from kpnet.utils.imgproc import PixelCoord, pixel_in_bbox
from kpnet.utils.transformations import quaternion_matrix
from kpnet.dataHandling.keypoint_db_entry import KeypointDBEntry
from kpnet.dataHandling.keypoint_db import KeypointDB, KeypointDBConfig
from kpnet.dataHandling.keypoint_loader import KeypointDatasetConfig, ProcessedEntry, KeypointDataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import kpnet.config.parameter as parameter
from kpnet.network.resnetNoStage import ResnetNoStage
from kpnet.network.resnetBackbone import ResnetConfig, initialize_backbone_from_modelzoo
import kpnet.network.predict as predict
from kpnet.network.losses import weighted_mse_loss, weighted_l1_loss
import time

In [2]:
kpDbConfig = KeypointDBConfig()
kpDbConfig.pdc_data_root = "/home/stanlew/src/kpam/data/"
#kpDbConfig.config_file_path = "/home/stanlew/src/kpam/mankey_recreate/kpnet/config/mugs_test_single.txt"
kpDbConfig.config_file_path = "/home/stanlew/src/kpam/mankey_recreate/kpnet/config/mugs_20190215.txt"
kpDbConfig.config_file_path = "/home/stanlew/src/kpam/mankey_recreate/kpnet/config/mugs_20190221.txt"
kpDbConfig.keypoint_yaml_name = "mug_3_keypoint_image.yaml"

In [3]:
kpDb_test = KeypointDB(kpDbConfig)

Keypoint DB Processing:  /home/stanlew/src/kpam/data/logs_proto/2019-02-21-02-53-49


/home/stanlew/src/kpam/mankey_recreate/kpnet/dataHandling/keypoint_db.py:125: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  keypoint_yaml_data = yaml.load(keypoint_yaml_file_handle)


Keypoint DB Processing:  /home/stanlew/src/kpam/data/logs_proto/2019-02-21-02-34-07
Keypoint DB Processing:  /home/stanlew/src/kpam/data/logs_proto/2019-02-21-02-43-12
Keypoint DB Processing:  /home/stanlew/src/kpam/data/logs_proto/2019-02-21-02-09-39
Keypoint DB Processing:  /home/stanlew/src/kpam/data/logs_proto/2019-02-21-03-09-14
Keypoint DB Processing:  /home/stanlew/src/kpam/data/logs_proto/2019-02-21-03-29-44
Keypoint DB Processing:  /home/stanlew/src/kpam/data/logs_proto/2019-02-21-02-59-15
Keypoint DB Processing:  /home/stanlew/src/kpam/data/logs_proto/2019-02-21-02-05-10
Keypoint DB Processing:  /home/stanlew/src/kpam/data/logs_proto/2019-02-21-01-41-16
Keypoint DB Processing:  /home/stanlew/src/kpam/data/logs_proto/2019-02-21-03-40-17
Keypoint DB Processing:  /home/stanlew/src/kpam/data/logs_proto/2019-02-21-01-59-28
Keypoint DB Processing:  /home/stanlew/src/kpam/data/logs_proto/2019-02-21-03-19-28
Keypoint DB Processing:  /home/stanlew/src/kpam/data/logs_proto/2019-02-21-0

In [4]:
kpLoaderConfig = KeypointDatasetConfig()
kpLoaderConfig.network_in_patch_width = 256
kpLoaderConfig.network_in_patch_height = 256
kpLoaderConfig.network_out_map_width = 64
kpLoaderConfig.network_out_map_height = 64
kpLoaderConfig.image_database_list.append(kpDb_test)
kpLoaderConfig.is_train = False

In [5]:
kpDatasetLoader_test = KeypointDataset(kpLoaderConfig)

In [6]:
networkConfig = ResnetConfig()

networkConfig.num_keypoints = 3
networkConfig.image_channels = 4
networkConfig.depth_per_keypoint = 2
networkConfig.num_layers = 34

In [7]:
network_weights_file = "/home/stanlew/src/kpam/mankey_recreate/training_checkpoints_all_data/checkpoint-135.pth"
#network_weights_file = "/home/stanlew/src/kpam/models/kpam_authors_135ckpt.pth"

In [8]:
network = ResnetNoStage(networkConfig)
network.load_state_dict(torch.load(network_weights_file))
network.cuda()
network.eval()

ResnetNoStage(
  (backbone_net): ResNetBackbone(
    (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, aff

In [9]:
loader_test = DataLoader(dataset=kpDatasetLoader_test, batch_size=25, shuffle=True, num_workers=8)

In [10]:
test_error_xy = 0
test_error_depth = 0

for idx, data in enumerate(loader_test):
    # Get the data
    image = data[parameter.rgbd_image_key]
    keypoint_xy_depth = data[parameter.keypoint_xyd_key]
    keypoint_weight = data[parameter.keypoint_validity_key]

    # Upload to GPU
    image = image.cuda()
    keypoint_xy_depth = keypoint_xy_depth.cuda()
    keypoint_weight = keypoint_weight.cuda()

    # predict
    raw_pred = network(image)
    prob_pred = raw_pred[:, 0:networkConfig.num_keypoints, :, :]
    depthmap_pred = raw_pred[:, networkConfig.num_keypoints:, :, :]
    heatmap = predict.heatmap_from_predict(prob_pred, networkConfig.num_keypoints)
    _, _, heatmap_height, heatmap_width = heatmap.shape

    # Compute the coordinate
    coord_x, coord_y = predict.heatmap2d_to_normalized_imgcoord_gpu(heatmap, networkConfig.num_keypoints)
    depth_pred = predict.depth_integration(heatmap, depthmap_pred)

    # Concantate them
    xy_depth_pred = torch.cat((coord_x, coord_y, depth_pred), dim=2)

    # Log info
    xy_error = float(weighted_l1_loss(xy_depth_pred[:, :, 0:2], keypoint_xy_depth[:, :, 0:2], keypoint_weight[:, :, 0:2]).item())
    depth_error = float(weighted_l1_loss(xy_depth_pred[:, :, 2], keypoint_xy_depth[:, :, 2], keypoint_weight[:, :, 2]).item())
    if idx % 100 == 0:
        batch_group_end = time.time()
        print('Iteration %d in testing epoch' % idx)
        print('The averaged pixel error is (pixel in 256x256 image): ', 256 * xy_error / len(xy_depth_pred))
        print('The averaged depth error is (mm): ', 256 * depth_error / len(xy_depth_pred))
        batch_group_start = batch_group_end

    # Update info
    test_error_xy += float(xy_error)
    test_error_depth += float(depth_error)
    
print('The testing averaged pixel error is (pixel in 256x256 image): ', 256 * test_error_xy / len(kpDatasetLoader_test))
print('The testing averaged depth error is (mm): ', kpLoaderConfig.depth_image_scale * test_error_depth / len(kpDatasetLoader_test))

Iteration 0 in testing epoch
The averaged pixel error is (pixel in 256x256 image):  0.12211732864379883
The averaged depth error is (mm):  0.2660479354858398
Iteration 100 in testing epoch
The averaged pixel error is (pixel in 256x256 image):  0.12232924461364746
The averaged depth error is (mm):  0.23876176834106444
Iteration 200 in testing epoch
The averaged pixel error is (pixel in 256x256 image):  0.13427621841430665
The averaged depth error is (mm):  0.2560139846801758
Iteration 300 in testing epoch
The averaged pixel error is (pixel in 256x256 image):  0.09542879104614257
The averaged depth error is (mm):  0.29747581481933594
Iteration 400 in testing epoch
The averaged pixel error is (pixel in 256x256 image):  0.11356779098510743
The averaged depth error is (mm):  0.2589230918884277
Iteration 500 in testing epoch
The averaged pixel error is (pixel in 256x256 image):  0.11466893196105957
The averaged depth error is (mm):  0.2035700798034668
Iteration 600 in testing epoch
The avera